In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/MyDrive/ift6289-project/
!ls

Mounted at /content/drive
/content/drive/MyDrive/ift6289-project
'Copy of ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'Copy of ift6289-project-mt5-base-ENtoBN.ipynb'
 gemma_7b_finetune_template.ipynb
 hasan-etal-2020-low
 hasan-etal-2020-low.tar.bz2
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-ENtoBN.ipynb'
'ift6289-project-facebook nllb-200-distilled-600M.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-bn-en.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-en-mul.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-mul-en.ipynb'
'ift6289-project-meta-llama Llama-2-7b-hf.ipynb'
 ift6289-project-mt5-base-BNtoEN.ipynb
 ift6289-project-mt5-base-ENtoBN.ipynb
 ift6289-project-umt5-base-BNtoEN.ipynb
 ift6289-project-umt5-base-ENtoBN.ipynb
 logs
 results


In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install transformers
!pip install ml_things
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 47.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.7 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=793ebfea500d38dbc67d

In [17]:
from transformers import set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline, MarianMTModel, MarianTokenizer
from datasets import Dataset, load_dataset, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm  # tqdm is great for progress bars
import sacrebleu
import numpy as np

In [18]:
set_seed(42)

dataset = load_dataset("csebuetnlp/BanglaNMT")
# print(dataset)
# print(dataset['train'][0])
# print(dataset['train'][1])
# print(dataset['train'][2])
# print(dataset['train'][0]['bn'])
# print(dataset['train'][0]['en'])

training_subset_size = 100
dataset["train"] = dataset["train"].select(range(training_subset_size))

In [19]:
# Specify the model checkpoint
# model_checkpoint = "facebook/nllb-200-distilled-600M"
# model_checkpoint = "Helsinki-NLP/opus-mt-bn-en"
model_checkpoint = "Helsinki-NLP/opus-mt-en-mul"
# model_checkpoint = "t5-base"
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [20]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['en'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    # Prepare the labels (Bengali translations)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['bn'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 597
    })
})


In [21]:
model = MarianMTModel.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
# Load both BLEU and SacreBLEU metrics
bleu_metric = load_metric('bleu')
sacrebleu_metric = load_metric('sacrebleu')

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # # Compute BLEU score
    # bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[decoded_labels])
    # Compute SacreBLEU score
    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare result dictionary
    results = {
        # "bleu": bleu_result['score'],
        "sacrebleu": sacrebleu_result['score']
    }

    return results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the 

In [23]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    # report_to="none",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# # Pre-training test results
baseline_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Baseline Test Metrics:", baseline_test_results.metrics)
baseline_validation_results = trainer.evaluate()
print("Baseline Validation Metrics:", baseline_validation_results)

# Fine-tune the model
print("\nFine-tuning the model...")
trainer.train()
print("Model fine-tuning complete.")

Baseline Test Metrics: {'test_loss': 11.982484817504883, 'test_sacrebleu': 0.02807164926570254, 'test_runtime': 166.887, 'test_samples_per_second': 5.992, 'test_steps_per_second': 0.378}


Baseline Validation Metrics: {'eval_loss': 11.816288948059082, 'eval_sacrebleu': 0.02970467902168954, 'eval_runtime': 98.7811, 'eval_samples_per_second': 6.044, 'eval_steps_per_second': 0.385}

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Sacrebleu
1,0.568100,0.580774,9.856759
2,0.478100,0.523479,12.122379
3,0.449400,0.507038,12.611845


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Model fine-tuning complete.


In [24]:
# Evaluate the model
validation_results = trainer.evaluate()
print("Final validation Metrics", validation_results)

Final validation Metrics {'eval_loss': 0.5070382356643677, 'eval_sacrebleu': 12.61184513277777, 'eval_runtime': 56.0086, 'eval_samples_per_second': 10.659, 'eval_steps_per_second': 0.678, 'epoch': 3.0}


In [25]:
# Calculate scores on the test set
final_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Final Test Metrics:", final_test_results.metrics)

Final Test Metrics: {'test_loss': 0.4939543604850769, 'test_sacrebleu': 12.376106483354917, 'test_runtime': 91.9399, 'test_samples_per_second': 10.877, 'test_steps_per_second': 0.685}


In [26]:
# Decode the predictions and labels
decoded_predictions = tokenizer.batch_decode(final_test_results.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(final_test_results.label_ids, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")

Sample Predictions and Labels:

Prediction 1: যুক্তরাজ্যের বাংলাদেশের হাই কমিশনার সাইদা মুনা তাসনিন এবং লন্ডনে বাংলাদেশ মিশনের সিনিয়র কর্মকর্তা বিমানবন্দরে তাকে দেখা যায়।
Label 1: বিমানবন্দরে যুক্তরাজ্যে নিযুক্ত বাংলাদেশ হাইকমিশনার সাঈদা মুনা তাসনীম ও লন্ডনে বাংলাদেশ মিশনের জ্যেষ্ঠ কর্মকর্তারা তাকে বিদায় জানান।

Prediction 2: সন্ত্রাসীদের কোন জাতি, ধর্ম ও দেশ নেই।
Label 2: সন্ত্রাসীদের কোনো বর্ণ, ধর্ম ও দেশ নেই।

Prediction 3: করনেল বিশ্ববিদ্যালয়ের অধ্যাপক ডেরিক ক্যাবেরা একটা বক্তৃতা লিখেছিলেন, "শ্রেণীবিদ্যালয়ের অন্ধকার পার্শ্ব"।
Label 3: কর্নেল বিশ্ববিদ্যালয়ের অধ্যাপক ডেরেক কাবেরা "শ্রেণিবদ্ধ চিন্তার অন্ধকার দিক" নামের একটি প্রবন্ধ রচনা করেছেন।

Prediction 4: তিনি বলেন যে, জনগণ প্রতি কেজি ৪৫ টি কে, ৫০ কিলোগ্রামে চাকার, টিকে ৫০ কিলোগ্রামে লেন্টিল এবং ৮৫ লিটারে সোয়েবিয়ান তেল কিনতে পারে।
Label 4: জনসাধারণ প্রতি কেজি পেঁয়াজ ৪৫ টাকায়, চিনি ৫০ টাকায়, মশুর ডাল ৫০ টাকায় ও প্রতি লিটার সয়াবিন তেল ৮৫ টাকায় কিনতে পারবে বলে জানান তিনি।

Prediction 5: স্রীপুর ফায়ার স্টেশনের দুটি ইউনি

# Test on SUPara-benchmark

In [27]:
supara_target_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_bn.txt"
supara_source_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_en.txt"

with open(supara_source_test_path, "r", encoding="utf-8") as f:
    supara_source_data = f.readlines()

with open(supara_target_test_path, "r", encoding="utf-8") as f:
    supara_target_data = f.readlines()

print(supara_source_data[0])
print(supara_target_data[0])

She likes red roses.

সে লাল গোলাপ পছন্দ করে।



In [31]:
supara_dataset = Dataset.from_dict({"en": supara_source_data, "bn": supara_target_data})
tokenized_supara_dataset = supara_dataset.map(preprocess_function, batched=True)
print(tokenized_supara_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['en', 'bn', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


In [32]:
final_test_results_supara = trainer.predict(tokenized_supara_dataset, metric_key_prefix="test")
print("Final test metrics SUPara:", final_test_results_supara.metrics)
print("Final test results SUPara:", final_test_results_supara)

Final test metrics SUPara: {'test_loss': 0.5837470889091492, 'test_sacrebleu': 10.735269278672352, 'test_runtime': 40.5408, 'test_samples_per_second': 12.333, 'test_steps_per_second': 0.789}
Final test results SUPara: PredictionOutput(predictions=array([[64109,     4,  8199, ..., 64109, 64109, 64109],
       [64109, 15405,  2876, ..., 64109, 64109, 64109],
       [64109, 26932,  6436, ..., 64109, 64109, 64109],
       ...,
       [64109,  6311,  3690, ..., 64109, 64109, 64109],
       [64109,  2479,  1342, ..., 64109, 64109, 64109],
       [64109,  6713,  9978, ..., 64109, 64109, 64109]]), label_ids=array([[    4,  8199,     4, ..., 64109, 64109, 64109],
       [10336, 16077,  2876, ..., 64109, 64109, 64109],
       [26932,  6436,  9004, ..., 64109, 64109, 64109],
       ...,
       [ 6311,  3690, 20613, ..., 64109, 64109, 64109],
       [ 6350,   716,  2312, ..., 64109, 64109, 64109],
       [ 6584,  1342,  2312, ..., 64109, 64109, 64109]]), metrics={'test_loss': 0.5837470889091492, '

In [33]:
# Decode the predicted sequences
decoded_predictions_supara = tokenizer.batch_decode(final_test_results_supara.predictions, skip_special_tokens=True)
labels_supara = np.where(final_test_results_supara.label_ids != -100, final_test_results_supara.label_ids, tokenizer.pad_token_id)
decoded_labels_supara = tokenizer.batch_decode(labels_supara, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions_supara[i]}")
    print(f"Label {i+1}: {decoded_labels_supara[i]}")

Sample Predictions and Labels:

Prediction 1: সে লাল রোজ পছন্দ করে।
Label 1: সে লাল গোলাপ পছন্দ করে।

Prediction 2: এটা আমাদের দেহকে দৃঢ় করে দেয়।
Label 2: ইহা আমাদের শরীরকে দৃঢ় করে।

Prediction 3: আমরা বাংলাদেশে বাস করছি।
Label 3: আমরা বাংলাদেশে বাস করি।

Prediction 4: মানুষের জীবন খুব ছোট।
Label 4: মানব জীবন অত্যন্ত সংক্ষিপ্ত।

Prediction 5: ট্রাফিক পুলিশের ভারী দায়িত্ব রয়েছে।
Label 5: ট্রাফিক পুলিশের অনেক দায়িত্ব থাকে।

Prediction 6: ক্রিকেট আমার প্রিয় খেলা।
Label 6: ক্রিকেট আমার প্রিয় খেলা।

Prediction 7: বিশাল সচেতনতা তৈরি করা উচিত।
Label 7: গণসচেতনতা সৃষ্টি করা উচিত।

Prediction 8: সম্ভবত কিছু সমস্যা হতে পারে।
Label 8: সেখানে কিছু সমস্যা থাকতে পারে।

Prediction 9: তার নাম কী?
Label 9: তার নাম কি?

Prediction 10: আমাদের কলেজ খুবই প্রাচীন।
Label 10: আমাদের কলেজ বহু প্রাচীন।


# Save predictions and labels

In [ ]:
# Combine predictions and labels into a DataFrame
predictions_df = pd.DataFrame({'Prediction': decoded_predictions_supara, 'Label': decoded_labels})

# Save the DataFrame to a CSV file
predictions_df.to_csv('helsinki_supara_predictions_en_bn.csv', index=False)

# Save the labels to a separate CSV file
labels_df = pd.DataFrame({'Label': decoded_labels_supara})
labels_df.to_csv('helsinki_supara_labels_en_bn.csv', index=False)